# SONY A6700 to Segment Anything Model (SAM) Pipeline

This notebook creates a pipeline for image segmentation:
1. Import necessary libraries and SAM
2. Take image with GPhoto2 Open-source camera control library
3. Augment and process image
4. Send image through SAM
5. Display segmented image

Requirements:
* Camera comptabile with GPhoto2
* Linux operating system
* GPU for SAM processing


## 1. Import necessary libraries and SAM

In [1]:
# install requirements
!pip install -r requirements.txt
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install -U numpy<2
!pip uninstall -y torch
!pip uninstall -y torchvision
!pip uninstall -y torchaudio
!pip install -U matplotlib
!pip install torch-2.5.0a0+872d972e41.nv24.08.17622132-cp310-cp310-linux_aarch64.whl
!wget raw.githubusercontent.com/pytorch/pytorch/5c6af2b583709f6176898c017424dc9981023c28/.ci/docker/common/install_cusparselt.sh 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-0327b9xb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-0327b9xb
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
/bin/bash: line 1: 2: No such file or directory
Found existing installation: torch 2.5.0a0+872d972e41.nv24.8
Uninstalling torch-2.5.0a0+872d972e41.nv24.8:
  Successfully uninstalled torch-2.5.0a0+872d972e41.nv24.8
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Processing ./torch-2.5.0a0+872d972e41.nv24.08.17622132-cp310-cp310-linux_aar

In [2]:
# import python libraries
import time
import os
import gphoto2 as gp 
import logging
import locale
import subprocess
import sys
import mimetypes
import datetime
import traceback
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torchvision

from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")
sam.to(device='cuda')
mask_generator = SamAutomaticMaskGenerator(sam)


/home/besslab-orin/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


ImportError: libcusparseLt.so.0: cannot open shared object file: No such file or directory

## 2. Take image with GPhoto2 Open-source camera control library

In [ ]:
# initialize camera object
camera = gp.Camera()
camera.init()

try:
    print("Capturing image....")
    file_path = camera.capture(gp.GP_CAPTURE_IMAGE)

    filename = file_path.name
    folder = file_path.folder

    # define where to store image, local directory in this case
    target_path = os.path.join(ohttps://developer.download.nvidia.com/compute/redist/jp/v51/pytorch/torch-1.14.0a0+44dac51c.nv23.01-cp38-cp38-linux_aarch64.whls.getcwd(), "test1.jpg")

    # download file from camera to host
    print(f"Saving image to {target_path}")
    camera_file = camera.file_get(folder, filename, gp.GP_FILE_TYPE_NORMAL)
    camera_file.save(target_path)

except gp.GPhoto2Error as ex:
    print(f"An error occured: {ex}")

finally:
    camera.exit()
    print("Success")


## 3. Augment and process image

In [ ]:
# load image into cv2
image_1 = cv2.imread(target_path)
image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15,15))
plt.imshow(image_1)
plt.axis('off')
plt.show()


## 4. Send image through SAM

In [ ]:
masks = mask_generator.generate(image_1)

## 5. Display segmented image

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35])
        img[m] = color_mask
    ax.imshow(img)

plt.figure(figsize=(15,15))
plt.imshow(image_1)
show_anns(masks)
plt.axis('off')
plt.show()